In [51]:
import collections
import math
import os
import random
import zipfile

In [52]:
from six.moves import urllib
from six.moves import xrange

In [53]:
import numpy as np
import tensorflow as tf

In [57]:
print(np.__version__)
print(tf.__version__)

DOWNLOADED_FILENAME = "SampleText.zip"

def maybe_downloaded(url_path, expected_bytes):
    if not os.path.exists(DOWNLOADED_FILENAME):
        filename, _ = urllib.request.urlretrieve(url_path, DOWNLOADED_FILENAME)
        
    statinfo = os.stat(DOWNLOADED_FILENAME)
    if statinfo.st_size == expected_bytes:
        print('Found and verified file from this path: ', url_path)
        print('Downloaded file: ', DOWNLOADED_FILENAME)
    else:
        print(statsinfo.st_size)

def read_words():
    with zipfile.ZipFile(DOWNLOADED_FILENAME) as f:
        firstfile = f.namelist()[0]
        filestring = tf.compat.as_str(f.read(firstfile))
        words = filestring.split()
        
    return words

URL_PATH = 'http://mattmahoney.net/dc/text8.zip'
FILESIZE = 31344016

maybe_downloaded(URL_PATH, FILESIZE)

1.14.2
1.6.0
Found and verified file from this path:  http://mattmahoney.net/dc/text8.zip
Downloaded file:  SampleText.zip


In [58]:
URL_PATH = 'http://mattmahoney.net/dc/text8.zip'
FILESIZE = 31344016

maybe_downloaded(URL_PATH, FILESIZE)

Found and verified file from this path:  http://mattmahoney.net/dc/text8.zip
Downloaded file:  SampleText.zip


In [62]:
vocabulary = read_words()

In [62]:
len(vocabulary)

17005207

In [63]:
vocabulary[:23]

['anarchism',
 'originated',
 'as',
 'a',
 'term',
 'of',
 'abuse',
 'first',
 'used',
 'against',
 'early',
 'working',
 'class',
 'radicals',
 'including',
 'the',
 'diggers',
 'of',
 'the',
 'english',
 'revolution',
 'and',
 'the']

In [72]:
def build_dataset(words, n_words):
    word_counts = [['UNKNOWN', -1]]
    
    counter = collections.Counter(words)
    word_counts.extend(counter.most_common(n_words - 1))
    
    dictionary = dict()
    
    for word, _ in word_counts:
        dictionary[word] = len(dictionary)
        
    word_indexes = list()
    
    unknown_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0 # dictinary['UNKNOWN']
            unknown_count += 1
            
        word_indexes.append(index)
        
    word_counts[0][1] = unknown_count
    
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return word_counts, word_indexes, dictionary, reversed_dictionary

In [73]:
VOCAVULARY_SIZE = 5000

word_counts, word_indexes, dictionary, reversed_dictionary = build_dataset(vocabulary, VOCAVULARY_SIZE)

In [74]:
word_counts[:10]

[['UNKNOWN', 2735459],
 ('the', 1061396),
 ('of', 593677),
 ('and', 416629),
 ('one', 411764),
 ('in', 372201),
 ('a', 325873),
 ('to', 316376),
 ('zero', 264975),
 ('nine', 250430)]

In [67]:
word_indexes[:10]

[0, 3081, 12, 6, 195, 2, 3134, 46, 59, 156]

In [76]:
import random

for key in random.sample(list(dictionary), 10):
    print(key, ":", dictionary[key])

like : 132
virus : 3333
philip : 2469
bah : 3034
liberalism : 4098
blocks : 3986
aware : 3747
acts : 1190
corporate : 2776
agree : 2989


In [85]:
for key in random.sample(list(reversed_dictionary), 10):
    print(key, ":", reversed_dictionary[key])

2778 : instructions
1188 : flight
819 : moved
4518 : inter
1518 : judaism
1692 : governments
1162 : microsoft
3376 : pain
3610 : controls
4691 : banking


In [88]:
del vocabulary

NameError: name 'vocabulary' is not defined

In [89]:
# Global index into words maintained across batches
global_index = 0

In [93]:
def generate_batch(words_indexes, batch_size, num_skips, skip_window):
    global global_index
    
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    lables = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    span = 2 * skip_window + 1
    
    buffer = collections.deque(maxlen=span)
    
    for _ in range(span):
        buffer.append(word_indexes[global_index])
        global_index = (global_index + 1) % len(word_indexes)
        
    for i in range(batch_size // num_skips):
        target = skip_window # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span -1)
                
            targets_to_avoid.append(target)
            
            batch[i * num_skips + j] = buffer[skip_window] # this is the input word
            lables[i * num_skips + j, 0] = buffer[target] # these are the context words
            
        buffer.append(word_indexes[global_index])
        global_index = (global_index + 1) % len(word_indexes)
    
    global_index = (global_index + len(word_indexes) - span) % len(word_indexes)
    
    return batch, lables

In [94]:
batch, labels = generate_batch(word_indexes, 10, 2, 5)

In [95]:
batch

array([  0,   0,   2,   2,   1,   1, 103, 103, 855, 855], dtype=int32)

In [96]:
labels

array([[742],
       [  3],
       [  1],
       [  0],
       [855],
       [  0],
       [  0],
       [  1],
       [  0],
       [  1]], dtype=int32)

In [98]:
for i in range(9):
    print(reversed_dictionary[batch[i]], ":", reversed_dictionary[labels[i][0]])

UNKNOWN : working
UNKNOWN : and
of : the
of : UNKNOWN
the : revolution
the : UNKNOWN
english : UNKNOWN
english : the
revolution : UNKNOWN


In [99]:
valid_size = 16
valid_window = 100

valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [104]:
batch_size = 128
embedding_size = 50
skip_window = 2
num_skips = 2

In [106]:
tf.reset_default_graph()

train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

In [107]:
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

In [108]:
embeddings = tf.Variable(tf.random_uniform([VOCAVULARY_SIZE, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [109]:
embeddings

<tf.Variable 'Variable:0' shape=(5000, 50) dtype=float32_ref>

In [110]:
embed

<tf.Tensor 'embedding_lookup:0' shape=(128, 50) dtype=float32>

In [113]:
weights = tf.Variable(tf.truncated_normal([VOCAVULARY_SIZE, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))

biases = tf.Variable(tf.zeros([VOCAVULARY_SIZE]))

hidden_out = tf.matmul(embed, tf.transpose(weights)) + biases

In [114]:
hidden_out

<tf.Tensor 'add:0' shape=(128, 5000) dtype=float32>

In [117]:
train_one_hot = tf.one_hot(train_labels, VOCAVULARY_SIZE)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hidden_out, labels=train_one_hot))

In [120]:
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)